In [2]:
import numpy as np
import random
import pickle
import gzip

In [3]:
def load_data():
    
    f = gzip.open('../../Resources/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

In [4]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

In [5]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [10]:
class Network(object):

    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):

        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def minibatch_gradient_descent(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
                
    def batch_gradient_descent(self, training_data, epochs, eta,
            test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            self.update_mini_batch(training_data, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
       
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
       
        return (output_activations-y)

In [9]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

### Data Loading

In [13]:
training_data, test_data, validation_data = load_data_wrapper()
training_data = [*training_data]
test_data = [*test_data]
len(training_data), len(test_data)

(50000, 10000)

### Hyperparams

In [14]:
epochs = 20
batch_size = 128
lr = 3.0

## Training the Network using mini-batch based Gradient Descent

In [17]:
%%time
nn = Network([784, 64, 10])
nn.minibatch_gradient_descent(training_data, epochs, batch_size, lr, test_data=test_data)

Epoch 0: 7642 / 10000
Epoch 1: 8475 / 10000
Epoch 2: 8781 / 10000
Epoch 3: 8920 / 10000
Epoch 4: 8992 / 10000
Epoch 5: 9068 / 10000
Epoch 6: 9107 / 10000
Epoch 7: 9160 / 10000
Epoch 8: 9190 / 10000
Epoch 9: 9224 / 10000
Epoch 10: 9247 / 10000
Epoch 11: 9279 / 10000
Epoch 12: 9288 / 10000
Epoch 13: 9313 / 10000
Epoch 14: 9322 / 10000
Epoch 15: 9331 / 10000
Epoch 16: 9346 / 10000
Epoch 17: 9358 / 10000
Epoch 18: 9376 / 10000
Epoch 19: 9384 / 10000
Wall time: 2min 38s


## Training the Network using batch-based Gradient Descent

In [18]:
%%time
nn = Network([784, 64, 10])
nn.batch_gradient_descent(training_data, epochs, lr, test_data=test_data)

Epoch 0: 1561 / 10000
Epoch 1: 1326 / 10000
Epoch 2: 1661 / 10000
Epoch 3: 1865 / 10000
Epoch 4: 1964 / 10000
Epoch 5: 2052 / 10000
Epoch 6: 2112 / 10000
Epoch 7: 2168 / 10000
Epoch 8: 2203 / 10000
Epoch 9: 2252 / 10000
Epoch 10: 2295 / 10000
Epoch 11: 2331 / 10000
Epoch 12: 2374 / 10000
Epoch 13: 2410 / 10000
Epoch 14: 2446 / 10000
Epoch 15: 2466 / 10000
Epoch 16: 2494 / 10000
Epoch 17: 2516 / 10000
Epoch 18: 2548 / 10000
Epoch 19: 2565 / 10000
Wall time: 2min 44s
